# Quality Assurance with RasCheck

This notebook demonstrates how to use the RasCheck module for validating HEC-RAS steady flow models.

In [ ]:
# =============================================================================
# DEVELOPMENT MODE TOGGLE
# =============================================================================
USE_LOCAL_SOURCE = False  # <-- TOGGLE THIS

if USE_LOCAL_SOURCE:
    import sys
    from pathlib import Path
    local_path = str(Path.cwd().parent)
    if local_path not in sys.path:
        sys.path.insert(0, local_path)
    print(f"📁 LOCAL SOURCE MODE: Loading from {local_path}/ras_commander")
else:
    print("📦 PIP PACKAGE MODE: Loading installed ras-commander")

# Import ras-commander
from ras_commander import HdfResultsPlan, RasCheck, RasCmdr, RasExamples, init_ras_project, ras

# Verify which version loaded
import ras_commander
print(f"✓ Loaded: {ras_commander.__file__}")

## Parameters

Configure these values to customize the notebook for your project.

In [ ]:
# =============================================================================
# PARAMETERS - Edit these to customize the notebook
# =============================================================================
from pathlib import Path

# Project Configuration
PROJECT_NAME = "Muncie"           # Example project to extract
RAS_VERSION = "6.6"               # HEC-RAS version (6.3, 6.5, 6.6, etc.)

# Output Configuration
OUTPUTS_DIR = Path("_outputs") / "800_quality_assurance_rascheck"  # Artifacts saved here

# Execution Settings
PLAN = "01"                       # Plan number to execute
NUM_CORES = 4                     # CPU cores for 2D computation
RUN_SUFFIX = "run"                # Suffix for run folder (e.g., Muncie_run)

# Create output directory
OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)
print(f"Outputs will be saved to: {OUTPUTS_DIR.absolute()}")


---

## Attribution

> **Based on FEMA cHECk-RAS**
>
> This module implements the validation checks described in FEMA's
> [cHECk-RAS](https://www.fema.gov/flood-maps/tutorials/check-ras) quality assurance tool
> for HEC-RAS 6.x models, using ras-commander's HDF function library.
>
> From FEMA's cHECk-RAS page:
>
> *"cHECk-RAS utilizes information generated by HEC-RAS (all versions through the latest
> version, 5.0.7). Note, cHECk-RAS is not compatible with the two-dimensional component
> of HEC-RAS 5.0.7."*
>
> This Python implementation extends these validation checks to work with modern HEC-RAS 6.x
> HDF-based outputs, providing equivalent functionality through the ras-commander library.

> **For Informational Purposes Only**
>
> Outputs from this library are for **informational purposes only** and should be
> validated by a qualified engineer. This tool does not replace professional engineering
> judgment or official FEMA review processes.

**NOTE**: This is an UNOFFICIAL Python implementation and is NOT affiliated with or endorsed by FEMA.

## Overview

The **RasCheck** module provides comprehensive quality assurance validation for HEC-RAS steady flow models. It implements the validation checks described in FEMA's cHECk-RAS using modern HDF-based data access for HEC-RAS 6.x models.

### Check Categories

| Check | Description | Key Validations |
|-------|-------------|------------------|
| **NT Check** | Manning's n and Transition | Roughness coefficients, contraction/expansion losses |
| **XS Check** | Cross Section Validation | Spacing, ineffective flow, reach lengths, levees |
| **Structure Check** | Bridges, Culverts, Weirs | Flow classification, coefficients, geometry |
| **Floodway Check** | Floodway Analysis | Surcharge limits, encroachment methods |
| **Profiles Check** | Multiple Profile Consistency | WSE ordering, discharge consistency |

### LLM Forward Verification Approach

This notebook follows **LLM Forward** principles for quality assurance:

1. **Visual Verification**: Every validation check produces reviewable outputs (HTML reports, DataFrames)
2. **Multiple Review Pathways**: 
   - Traditional engineering review (HTML reports with message IDs)
   - Visual inspection (DataFrame filtering and grouping)
   - Code audit trail (@log_call decorators track all validation operations)
3. **Professional Responsibility**: These checks support, but do not replace, professional engineering judgment
4. **Self-Documenting**: Export to CSV/HTML creates permanent audit trail of QA findings

### Quality Standards Reference

RasCheck implements validation checks aligned with:

- **[FEMA Guidelines and Standards for Flood Risk Analysis](https://www.fema.gov/flood-maps/guidance-partners/guidelines-standards)**
  - Appendix D: Guidance for Riverine Flooding Analyses and Mapping
  - Standards for model geometry, hydraulic parameters, and boundary conditions

- **[USACE EM 1110-2-1416: River Hydraulics](https://www.publications.usace.army.mil/USACE-Publications/Engineer-Manuals/)**
  - Chapter 10: Model Calibration and Verification
  - Guidance on Manning's n selection, cross section spacing, structure modeling

- **State/Local Floodplain Management Requirements**
  - State-specific floodway surcharge limits (IL: 0.1 ft, TX: 1.0 ft, etc.)
  - Local drainage district standards (where applicable)

### When to Use RasCheck

**Pre-Submittal QA** (before submitting to FEMA):
- Run RasCheck validation on final model
- Generate HTML report for submittal package
- Document how each ERROR/WARNING was addressed
- Include report in engineering analysis documentation

**Mid-Development QA** (during model development):
- Run checks iteratively as model is refined
- Use custom thresholds for project-specific requirements
- Filter by check type to focus on specific issues

**Post-Calibration QA** (after calibration complete):
- Verify model still meets QA standards after calibration adjustments
- Document any parameter changes that triggered new messages
- Ensure calibration didn't introduce geometry issues

## Import Required Modules

**Note**: The RasCheck module is currently in development. Ensure `USE_LOCAL_SOURCE = True` in the toggle cell above to load the local development version.

For detailed demonstrations of the new validation features, see:
- **301_advanced_structure_validation.ipynb** - Culvert and starting WSE validation
- **302_custom_workflows_and_standards.ipynb** - State-specific standards and custom workflows

In [6]:
from ras_commander import (
    RasExamples, init_ras_project, ras,
    RasCmdr, HdfResultsPlan
)
from ras_commander.check import (
    RasCheck, CheckResults, Severity,
    RasCheckReport, ReportMetadata,
    get_default_thresholds, create_custom_thresholds,
    get_state_surcharge_limit
)

import pandas as pd
from pathlib import Path

# Set pandas display options
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

ModuleNotFoundError: No module named 'ras_commander.check'

## Extract and Initialize Project

We'll use the **Bald Eagle Creek** example project which contains both steady and unsteady flow plans. RasCheck validates steady flow models.

In [ ]:
# Extract the Bald Eagle Creek example project
project_path = RasExamples.extract_project(PROJECT_NAME, output_path=OUTPUTS_DIR.parent / "example_projects")
print(f"Extracted project to: {project_path}")

In [ ]:
# Initialize the project
init_ras_project(project_path, RAS_VERSION)
print(f"Initialized project: {ras.project_name}")

In [ ]:
# View available plans
print("Available plans:")
ras.plan_df[['plan_number', 'Plan Title', 'flow_type']]

## Run the Steady Flow Plan

RasCheck requires computed results in the HDF file. We'll run the steady flow plan (Plan 02) if results don't exist.

In [ ]:
# Use the steady flow plan
plan_number = "02"

# Execute the plan (skip if results already exist)
print(f"Running Plan {plan_number}...")
success = RasCmdr.compute_plan(plan_number, skip_existing=True)
if success:
    print(f"Plan {plan_number} ready for validation")
else:
    print(f"Plan {plan_number} execution failed - check HEC-RAS installation")

In [ ]:
# Verify this is a steady state plan
is_steady = HdfResultsPlan.is_steady_plan(plan_number)
print(f"Is Plan {plan_number} a steady state plan? {is_steady}")

# Get profile names
profiles = HdfResultsPlan.get_steady_profile_names(plan_number)
print(f"\nFound {len(profiles)} profiles: {profiles}")

---

## Running RasCheck Validation

### Option 1: Run All Checks at Once

The simplest approach is to run all checks in one call:

In [ ]:
# Run all validation checks
results = RasCheck.run_all(plan_number)

# View summary
print(f"Validation Results:")
print(f"  Total Messages: {len(results.messages)}")
print(f"  Errors: {results.get_error_count()}")
print(f"  Warnings: {results.get_warning_count()}")

### Verification Checklist

After running validation checks, review the following:

**Check Results Summary**:
- [ ] Total ERROR messages < 10 (or zero for final submittal)
- [ ] All CRITICAL issues addressed (model cannot be submitted with CRITICAL errors)
- [ ] WARNING messages reviewed and documented (explain why acceptable)

**Success Criteria**:
- **FEMA Submittal**: Zero ERROR or CRITICAL messages
- **Development QA**: <20 WARNING messages, zero CRITICAL
- **Post-Calibration**: No new ERROR messages introduced by calibration changes

**Audit Trail Documentation**:
1. Save HTML report to `qa_reports/{project_name}_{date}_rascheck.html`
2. Export CSV of all messages for tracking: `qa_reports/{project_name}_{date}_messages.csv`
3. Document resolution of each ERROR/WARNING in engineering analysis
4. Include report in submittal package (FEMA Appendix L: Model Certification)

**Common Issues to Address**:
- Manning's n outside typical ranges (see EM 1110-2-1416 Table 7-6)
- Cross section spacing > 500 ft in areas of changing topography
- Ineffective flow areas overlapping channel
- Bridge/culvert expansion coefficients > 0.5
- Floodway surcharge exceeding state limits

### Option 2: Run Individual Checks

You can also run specific checks independently for more control:

In [ ]:
# Get HDF paths for individual checks
plan_row = ras.plan_df[ras.plan_df['plan_number'] == plan_number].iloc[0]
plan_hdf = Path(plan_row['HDF_Results_Path'])
geom_hdf = Path(plan_row['Geom Path']).with_suffix('.hdf')

print(f"Plan HDF: {plan_hdf}")
print(f"Geom HDF: {geom_hdf}")

In [ ]:
# Run NT Check (Manning's n and Transition coefficients)
nt_results = RasCheck.check_nt(geom_hdf)
print(f"NT Check: {len(nt_results.messages)} messages")
print(f"  Errors: {nt_results.get_error_count()}")
print(f"  Warnings: {nt_results.get_warning_count()}")

---

## Analyzing Results

### Convert Results to DataFrame

In [ ]:
# Convert all messages to DataFrame for analysis
df = results.to_dataframe()

if not df.empty:
    print(f"Results DataFrame shape: {df.shape}")
    print(f"\nColumns: {list(df.columns)}")
    display(df.head(10))
else:
    print("No validation messages generated")

### Filter Messages by Severity

In [ ]:
# Get only ERROR messages
errors = results.filter_by_severity(Severity.ERROR)
print(f"Found {len(errors)} ERROR messages:")

for msg in errors[:5]:  # Show first 5
    print(f"  [{msg.message_id}] RS {msg.station}: {msg.message}")

In [ ]:
# Get only WARNING messages
warnings = results.filter_by_severity(Severity.WARNING)
print(f"Found {len(warnings)} WARNING messages:")

for msg in warnings[:5]:  # Show first 5
    print(f"  [{msg.message_id}] RS {msg.station}: {msg.message}")

### Filter Messages by Check Type

In [ ]:
# Group messages by check type
if not df.empty:
    check_summary = df.groupby(['check_type', 'severity']).size().unstack(fill_value=0)
    print("Messages by Check Type and Severity:")
    display(check_summary)

In [ ]:
# Filter by specific check type
nt_msgs = results.filter_by_check_type("NT")
print(f"\nNT Check Messages ({len(nt_msgs)} total):")

# Show message ID distribution
if nt_msgs:
    msg_ids = [m.message_id for m in nt_msgs]
    print(pd.Series(msg_ids).value_counts())

### Filter Messages by Station

In [ ]:
# Get unique stations with issues
if not df.empty:
    stations_with_issues = df['station'].unique()
    print(f"Stations with issues: {len(stations_with_issues)}")
    print(f"\nFirst 10 stations: {list(stations_with_issues[:10])}")

In [ ]:
# Filter messages for a specific station
if not df.empty and len(stations_with_issues) > 0:
    sample_station = stations_with_issues[0]
    station_msgs = results.filter_by_station(sample_station)
    
    print(f"\nMessages for station {sample_station}:")
    for msg in station_msgs:
        print(f"  [{msg.severity.value}] {msg.message_id}: {msg.message}")

---

## Generating Reports

### HTML Report

Generate a comprehensive HTML report with all validation results:

In [ ]:
# Define output path for report
report_path = project_path / "ras_checker" / "validation_report.html"
report_path.parent.mkdir(parents=True, exist_ok=True)

# Generate HTML report
output_path = results.to_html(report_path)
print(f"HTML report generated: {output_path}")

### Using QA Reports in FEMA Submittal

**FEMA Submittal Requirements** ([Guidelines and Standards, Appendix D](https://www.fema.gov/flood-maps/guidance-partners/guidelines-standards)):

1. **Model Certification** (Appendix L):
   - Professional Engineer must certify model meets FEMA standards
   - RasCheck report provides supporting documentation
   - Document how all ERROR/WARNING messages were addressed

2. **Engineering Analysis** (required deliverable):
   - Include RasCheck HTML report in analysis appendix
   - Provide narrative explaining any WARNING messages
   - Reference specific FEMA/USACE guidance for parameter selection

3. **Quality Assurance Log**:
   - Date of QA check
   - RasCheck version used
   - Summary of ERROR/WARNING counts
   - Resolution documentation for each issue

**Report Interpretation**:

| Severity | Submittal Action | Example |
|----------|------------------|---------|
| **INFO** | No action needed | "Manning's n = 0.045 in typical range" |
| **WARNING** | Document in analysis | "XS spacing = 520 ft, acceptable for gradual terrain" |
| **ERROR** | Must fix before submittal | "Manning's n = 0.200, exceeds maximum (0.150)" |
| **CRITICAL** | Fix immediately | "HDF file corrupted, results invalid" |

**References for Parameter Justification**:
- Manning's n: [Chow (1959) Open Channel Hydraulics](https://www.ce.utexas.edu/prof/maidment/CE374K/Chow59.pdf)
- Cross section spacing: USACE EM 1110-2-1416, Section 10-3
- Expansion/contraction: [FHWA HDS-1 Bridge Hydraulics](https://www.fhwa.dot.gov/engineering/hydraulics/pubs/012063.pdf)

### HTML Report with Metadata

Add project metadata for more informative reports:

In [ ]:
# Create report with metadata
metadata = ReportMetadata(
    project_name=ras.project_name,
    project_path=project_path,  # Path object, not str
    plan_name="Steady Flow Run"
)

# Generate report with metadata
report_path_meta = project_path / "ras_checker" / "validation_report_with_metadata.html"
output_path = results.to_html(report_path_meta, metadata=metadata)
print(f"Report with metadata generated: {output_path}")

### Export to CSV

Export messages to CSV for further analysis in Excel or other tools:

In [ ]:
# Export to CSV
csv_path = project_path / "ras_checker" / "validation_messages.csv"

df = results.to_dataframe()
if not df.empty:
    df.to_csv(csv_path, index=False)
    print(f"CSV exported: {csv_path}")
    print(f"Exported {len(df)} messages")
else:
    print("No messages to export")

---

## Customizing Validation Thresholds

RasCheck uses configurable thresholds for all checks. You can customize these for project-specific requirements.

In [ ]:
# View default thresholds
defaults = get_default_thresholds()

print("Default Manning's n Thresholds:")
print(f"  Overbank: {defaults.mannings_n.overbank_min} - {defaults.mannings_n.overbank_max}")
print(f"  Channel:  {defaults.mannings_n.channel_min} - {defaults.mannings_n.channel_max}")

print("\nDefault Transition Coefficients:")
print(f"  Structures: Contraction={defaults.transitions.structure_contraction_max}, Expansion={defaults.transitions.structure_expansion_max}")
print(f"  Regular XS: Contraction={defaults.transitions.regular_contraction_max}, Expansion={defaults.transitions.regular_expansion_max}")

In [ ]:
# Create custom thresholds
custom_thresholds = create_custom_thresholds({
    # Stricter Manning's n limits
    'mannings_n.overbank_min': 0.035,
    'mannings_n.overbank_max': 0.150,
    'mannings_n.channel_min': 0.028,
    'mannings_n.channel_max': 0.080,
    
    # Custom reach length limits
    'reach_length.max_length_ft': 2000.0,
    'reach_length.length_ratio_max': 1.5,
})

print("Custom thresholds created")
print(f"  Custom overbank min: {custom_thresholds.mannings_n.overbank_min}")

In [ ]:
# Run checks with custom thresholds
custom_results = RasCheck.run_all(plan_number, thresholds=custom_thresholds)

print(f"\nResults with custom thresholds:")
print(f"  Total Messages: {len(custom_results.messages)}")
print(f"  Errors: {custom_results.get_error_count()}")
print(f"  Warnings: {custom_results.get_warning_count()}")

print(f"\nComparison with default thresholds:")
print(f"  Default: {len(results.messages)} messages")
print(f"  Custom:  {len(custom_results.messages)} messages")

---

## State-Specific Floodway Surcharge Limits

Different states have different maximum surcharge limits for floodway analysis:

In [ ]:
# View state-specific surcharge limits
print("State-Specific Surcharge Limits:")
print("="*40)

states = ['TX', 'IL', 'WI', 'MN', 'NJ', 'MI', 'IN', 'OH']
for state in states:
    limit = get_state_surcharge_limit(state)
    print(f"  {state}: {limit:.2f} ft")

In [ ]:
# Example: Using Illinois standards (stricter 0.1 ft surcharge)
il_limit = get_state_surcharge_limit('IL')
print(f"Illinois surcharge limit: {il_limit} ft")

# You would use this when running floodway checks:
# results = RasCheck.run_all(
#     plan_number,
#     floodway_profile='Floodway',
#     surcharge=il_limit
# )

---

## Understanding Message IDs

RasCheck uses a standardized message ID format:

```
{CHECK_TYPE}_{CATEGORY}_{NUMBER}{SUFFIX}
```

Examples:
- `NT_RC_01L` - NT Check, Roughness Coefficient, Message 01, Left side
- `XS_DT_02R` - XS Check, Distance/Travel, Message 02, Right side
- `BR_TF_04` - Bridge, Type Flow, Message 04

In [ ]:
# Analyze message ID distribution
if not df.empty:
    print("Message ID Distribution:")
    print("="*50)
    msg_counts = df['message_id'].value_counts()
    print(msg_counts.head(15))

---

## Summary

This notebook demonstrated how to use RasCheck for HEC-RAS model validation following LLM Forward principles:

### What You Learned

1. **Running Validation Checks**:
   - `RasCheck.run_all()` for comprehensive validation
   - Individual check methods for targeted QA
   - Custom thresholds for project-specific requirements

2. **Analyzing Results**:
   - Filter by severity (ERROR, WARNING, INFO)
   - Group by check type (NT, XS, Structure, etc.)
   - Filter by station for site-specific issues

3. **Generating Audit Trail**:
   - HTML reports for submittal packages
   - CSV exports for issue tracking
   - Metadata documentation for project context

4. **Quality Standards Compliance**:
   - FEMA Guidelines and Standards alignment
   - State-specific floodway surcharge limits
   - USACE EM 1110-2-1416 parameter guidance

### Next Steps

**For FEMA Submittal**:
1. Run RasCheck on final calibrated model
2. Generate HTML report with project metadata
3. Document resolution of all ERROR/WARNING messages
4. Include report in engineering analysis (Appendix)
5. Reference in Model Certification (PE seal)

**For Advanced Validation**:
- **801_advanced_structure_validation.ipynb** - Culvert and starting WSE checks
- **802_custom_workflows_and_standards.ipynb** - State-specific workflows

**For Model Development**:
- Use RasCheck iteratively during development
- Export CSV for issue tracking in project management tools
- Create custom thresholds for project-specific criteria

### Key Classes and Methods

| Class/Method | Description | When to Use |
|--------------|-------------|-------------|
| `RasCheck.run_all()` | Run all validation checks | Final QA before submittal |
| `RasCheck.check_nt()` | Manning's n and transition checks | After roughness calibration |
| `RasCheck.check_xs()` | Cross section validation | After geometry development |
| `RasCheck.check_structures()` | Bridge/culvert/weir checks | After structure modeling |
| `RasCheck.check_floodways()` | Floodway surcharge validation | Floodway certification |
| `RasCheck.check_profiles()` | Multiple profile consistency | After running multiple profiles |
| `CheckResults.to_html()` | Generate HTML report | Submittal documentation |
| `CheckResults.to_dataframe()` | Convert to pandas DataFrame | Custom analysis/filtering |
| `create_custom_thresholds()` | Create custom validation criteria | Project-specific requirements |
| `get_state_surcharge_limit()` | Get state-specific floodway limits | Multi-state projects |

### Professional Responsibility

**IMPORTANT**: RasCheck outputs are for **informational purposes only**. These checks:
- Support, but do not replace, professional engineering judgment
- Should be reviewed by a licensed Professional Engineer
- Must be validated against project-specific requirements
- Do not constitute official FEMA review or approval

Quality assurance is a professional responsibility that requires:
- Engineering expertise in floodplain hydraulics
- Knowledge of FEMA/USACE standards and guidelines
- Understanding of local drainage district requirements
- Proper documentation and certification

### References

**FEMA Resources**:
- [FEMA Guidelines and Standards for Flood Risk Analysis](https://www.fema.gov/flood-maps/guidance-partners/guidelines-standards)
  - Appendix D: Guidance for Riverine Flooding Analyses and Mapping
  - Appendix L: Model Certification Requirements
- [FEMA cHECk-RAS](https://www.fema.gov/flood-maps/tutorials/check-ras) (original validation tool)

**USACE Resources**:
- [EM 1110-2-1416: River Hydraulics](https://www.publications.usace.army.mil/USACE-Publications/Engineer-Manuals/)
  - Chapter 7: Roughness Coefficients (Manning's n tables)
  - Chapter 10: Model Calibration and Verification
- [FHWA HDS-1: Highway Bridge Hydraulics](https://www.fhwa.dot.gov/engineering/hydraulics/pubs/012063.pdf)

**Technical References**:
- Chow, V.T. (1959) *Open Channel Hydraulics* (Manning's n selection)
- Arcement & Schneider (1989) *Guide for Selecting Manning's Roughness Coefficients* (USGS WSP 2339)

### Support

For RasCheck documentation and examples, see:
- RasCheck API reference in ras-commander docs
- Example notebooks 801-802 for advanced validation workflows
- GitHub issues for bug reports or feature requests